In [1]:
search_for = 139
start_from = 0
threads = 10
thread = 9
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.11930561065673828
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 376006121
type: [1, 1, 1, 1, 139] 139
cases of this type: 373301041
100000 11.322302538523328
200000 17.236830563263293
300000 21.996202301733913
400000 19.77996422015559
500000 21.77817735158349
600000 19.715073043334755
700000 21.003553594866073
800000 18.538900189244842
900000 23.88851280928191
1000000 18.980695173605447
1100000 21.864846924897897
1200000 19.470083712023673
1300000 20.41374938134249
1400000 5.621025790057896
1500000 18.971532723725172
1600000 21.715414048135532
1700000 19.994614667514934
1800000 18.009862909316336
1900000 17.449417994119347
2000000 22.519131524328174
2100000 24.492872978459687
2200000 25.898151013748343
2300000 31.031941137661956
2400000 36.50864236018599
2500000 37.461984965517296
2600000 39.04399379815119
2700000 18.396091653011187
2800000 24.440781004381453
2900000 40.92887204110631
3000000 

29700000 35.16657982047505
29800000 12.431704786573816
29900000 31.647149056119066
30000000 32.66457336825066
30100000 33.80728993651728
30200000 32.37626656284067
30300000 32.23743754465423
30400000 33.68254723226312
30500000 32.054776048746604
30600000 31.869051287734486
30700000 31.47162843654857
30800000 30.400318833648317
30900000 15.604111095621192
31000000 25.119622917225108
31100000 29.609315533597478
31200000 32.42313486452635
31300000 30.420406247634272
31400000 33.425448431815916
31500000 30.521575999579333
31600000 33.01156144379884
31700000 31.326824719616393
31800000 34.029801745457725
31900000 31.632563811749574
32000000 23.953654142217562
32100000 22.973058115659814
32200000 34.742778210121315
32300000 33.38039438677241
32400000 34.98768468539068
32500000 14.046717022182925
32600000 31.509566167026293
32700000 33.58151693397211
32800000 35.91393122532954
32900000 34.03945920512902
33000000 32.83333520680403
33100000 32.87501303440769
33200000 32.41475398237571
33300000 

59600000 16.109920383972252
59700000 29.75904969591527
59800000 30.573759512741592
59900000 28.64408877571486
60000000 29.7781960983236
60100000 28.727548392537678
60200000 28.561191574940047
60300000 27.084649444765606
60400000 24.34276317185756
60500000 12.268330776420054
60600000 28.480316415544706
60700000 27.450138425296537
60800000 29.005958754468
60900000 28.083912159712696
61000000 30.082448989551683
61100000 28.710845821991803
61200000 30.038403612652328
61300000 28.826758733877014
61400000 12.743163629960714
61500000 31.083502223282125
61600000 30.32440238815674
61700000 30.760669275717795
61800000 31.431313153521646
61900000 32.19221958705075
62000000 30.31341725836324
62100000 31.44803472820793
62200000 26.47174317571026
62300000 15.186975708823892
62400000 29.679660860249964
62500000 29.617183906344344
62600000 29.63098859996303
62700000 31.616463502214973
62800000 28.22786795447561
62900000 27.392678990645813
63000000 27.242239295107368
63100000 19.700035608259018
6320000

89400000 23.576233951603413
89500000 26.20899500709781
89600000 25.781006532133777
89700000 27.365898544960377
89800000 25.81646339655234
89900000 26.199092734251646
90000000 8.57558177001261
90100000 24.515683492932162
90200000 25.81914133746732
90300000 26.157335924833273
90400000 25.506490863889628
90500000 26.171846523008867
90600000 26.56943597621776
90700000 11.115634519540231
90800000 27.367519330239027
90900000 28.45720464853529
91000000 29.14283412133869
91100000 28.468104140156093
91200000 29.7603322321548
91300000 31.572906111456998
91400000 31.2109283411511
91500000 30.003529405439725
91600000 30.263418778944107
91700000 29.968755386453566
91800000 29.640612994271045
91900000 29.537327212554757
92000000 14.08130124360798
92100000 21.990553685446972
92200000 27.34628552892542
92300000 27.350922751857546
92400000 25.916401251689635
92500000 26.84616484545064
92600000 27.891317836927684
92700000 4.623803962713703
92800000 28.447065241361027
92900000 27.73585143180403
93000000 

118500000 26.771095959492893
118600000 26.205952147450176
118700000 26.164846906997656
118800000 24.55491595011766
118900000 24.59657177550836
119000000 21.00393230254974
119100000 10.996324133635865
119200000 23.18238381261283
119300000 23.55450435982397
119400000 23.869129724443297
119500000 17.302313852695647
119600000 13.862103380490481
119700000 24.41405238363499
119800000 23.665923514536118
119900000 23.951720573834194
120000000 15.194719242892608
120100000 18.16305887810462
120200000 24.515235752716766
120300000 24.219473122720988
120400000 25.440443914810544
120500000 24.889772969203698
120600000 26.379212559163673
120700000 26.059971391439355
120800000 26.18987808347445
120900000 26.000023056023895
121000000 26.206735871028734
121100000 25.690895675649518
121200000 25.888354188991872
121300000 25.519834778290498
121400000 25.728901070456267
121500000 23.719413442744113
121600000 23.075100836385978
121700000 21.40555430879371
121800000 9.149016623645453
121900000 23.38228561847

...

233100000 13.600819297140053
233200000 13.02690043630563
233300000 13.621315595235089
233400000 13.928395295310835
233500000 14.741415650770653
233600000 13.858870845630895
233700000 14.605376791697642
233800000 14.372440676453952
233900000 15.389571860689951
234000000 14.09363728208497
234100000 14.511248345119972
234200000 12.982232962658102
234300000 13.48839149683628
234400000 12.04992704489611
234500000 13.423141696513884
234600000 12.26022737160243
234700000 4.697406042840781
234800000 12.90635618597128
234900000 12.52589347233254
235000000 7.0334023985980485
235100000 9.024976334088995
235200000 12.972027039938895
235300000 12.232053611400017
235400000 4.459245792527421
235500000 12.854217320010374
235600000 12.721767836116557
235700000 12.99021453137457
235800000 13.71784949998955
235900000 12.92393045449833
236000000 14.036948521120143
236100000 13.940855369998838
236200000 14.766584543166221
236300000 13.626899200728072
236400000 14.729533720070384
236500000 13.18967995619981

261700000 10.674890710655275
261800000 10.827047358500499
261900000 1.756447783221466
262000000 10.614711317484693
262100000 10.878266409743258
262200000 10.024810437064545
262300000 10.689331489248557
262400000 6.063808239672473
262500000 8.28722106681563
262600000 10.0023203112099
262700000 11.404664106559505
262800000 10.7249482533174
262900000 12.381136182944983
263000000 10.694547388953946
263100000 11.996658876041636
263200000 11.083508105662926
263300000 12.3132827539964
263400000 11.025471459995108
263500000 11.361102547489777
263600000 10.795495383404084
263700000 11.612714638347898
263800000 11.175193331185572
263900000 10.715108355535214
264000000 4.96964685649982
264100000 8.958493666658804
264200000 10.44049876963503
264300000 9.862352981509524
264400000 10.465484468856951
264500000 9.294782018983549
264600000 3.441872117324443
264700000 10.10361348253435
264800000 10.7835696757337
264900000 9.355897127250495
265000000 10.479210712512199
265100000 6.196724460395173
2652000

290900000 8.79554887405369
291000000 7.44810520932301
291100000 8.560136009608467
291200000 7.74787554134232
291300000 8.380283292368178
291400000 4.352432911531586
291500000 6.239241259502918
291600000 7.718287899857769
291700000 8.397918375710692
291800000 7.637628057077485
291900000 8.363689617748015
292000000 8.175240220302799
292100000 8.725460134100283
292200000 2.6309017587072945
292300000 8.871298844956293
292400000 8.98575112611469
292500000 8.591145267486016
292600000 8.90847512039811
292700000 8.710553089458845
292800000 9.11829384727125
292900000 8.720304706294762
293000000 9.234910093021067
293100000 8.282152372637821
293200000 8.998893482809748
293300000 5.720403858650613
293400000 5.7870549615703615
293500000 7.525586430964818
293600000 8.499731870543073
293700000 6.8967759259531585
293800000 8.032006759929041
293900000 7.191253422360046
294000000 8.048448014645658
294100000 3.09885315424637
294200000 6.649089936089749
294300000 7.248272188022821
294400000 7.878446516358

320200000 5.575953278591692
320300000 5.776573859188098
320400000 5.232559396545249
320500000 5.289760804735561
320600000 5.3026485655718565
320700000 4.983709429119791
320800000 5.035528628301148
320900000 4.48350468003762
321000000 1.8529361251279743
321100000 4.792003222401463
321200000 5.052007748700231
321300000 4.743042340267297
321400000 5.6633592290160895
321500000 5.15109284929778
321600000 5.481171780061516
321700000 5.10261800513322
321800000 5.7414023521943225
321900000 3.5132224457126875
322000000 3.7674653937984757
322100000 5.011769901500344
322200000 5.758157990756149
322300000 5.041851520700792
322400000 5.762633127746677
322500000 5.170007799266728
322600000 5.001001214445681
322700000 2.485733532322049
322800000 5.543953821524496
322900000 5.248208454346366
323000000 5.225918498382938
323100000 4.962568800542496
323200000 5.054075383096208
323300000 5.0320164508786105
323400000 4.664940676415889
323500000 4.784019505313228
323600000 3.6501005403882587
323700000 2.513

349200000 2.6778149207574415
349300000 1.7108433489783241
349400000 2.0943870640584925
349500000 2.571238671003177
349600000 2.6464976618398635
349700000 2.4654153654849984
349800000 2.723574433399047
349900000 2.349766426585672
350000000 2.616507383935481
350100000 2.352111935357875
350200000 2.6211894993985894
350300000 2.3509937456469014
350400000 2.466851656762792
350500000 0.9057499479191158
350600000 2.1024280197464664
350700000 2.3565368778960805
350800000 2.521532237689856
350900000 2.291765672420722
351000000 2.460923639256284
351100000 2.293881926500991
351200000 2.536954190662946
351300000 2.3314934919133408
351400000 2.4360730441691545
351500000 2.3535038308948137
351600000 2.578111657581064
351700000 0.5554344046029257
351800000 2.4591863744209883
351900000 2.4066408906856096
352000000 0.8698511112199002
352100000 2.4161438530300003
352200000 2.364478230409001
352300000 2.2911199416223176
352400000 2.260120417135624
352500000 2.3611404082448253
352600000 2.169164133871188
